DATA Cleaning

In [4]:
import pandas as pd

In [37]:
df = pd.read_parquet("hf://datasets/qwedsacf/story-generation/data/train-00000-of-00001-03560cf8eb66eee4.parquet")

In [1]:
import pandas as pd

df = pd.read_parquet("hf://datasets/juancopi81/orca-math-word-problems-140028_150030/data/train-00000-of-00001.parquet")

/Users/joanneliu/Desktop/AI4DM/H3M3/FlaskApp_Template/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/joanneliu/Desktop/AI4DM/H3M3/FlaskApp_Template/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import pandas as pd

df = pd.read_parquet("hf://datasets/bunnycore/Kind-Teacher/data/train-00000-of-00001.parquet")

In [9]:
def write_stories_to_txt(df, output_file):
    # Open the output file in write mode
    with open(output_file, 'w') as f:
        # Write each text entry on a new line
        for text in df['completion']:
            f.write(f"{text}\n")

    print(f"Problems have been written to {output_file}")

In [10]:
write_stories_to_txt(df, "context.txt")

Problems have been written to context.txt


RAG

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_community.document_loaders import TextLoader #load the document
from langchain_text_splitters import RecursiveCharacterTextSplitter #for creating chunks from the loaded document
from langchain_openai import OpenAIEmbeddings #for converting chunks into embeddings
from langchain_chroma import Chroma #database for stroring the embeddings

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import os
dir = os.getcwd()
db_dir = os.path.join(dir,"chroma_db")
print(db_dir)

/Users/joanneliu/Desktop/AI4DM/H3M3/FlaskApp_Template/chroma_db


Create vector DB

In [56]:
#Read the text content from the .txt file and load it as langchain document
loader = TextLoader('contextStories.txt')
document = loader.load()

In [11]:
loader = TextLoader('context.txt')
document = loader.load()

In [12]:
#Split the document into chunks using text splitters 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(document)

print("Document chunk info:\n")
print(f"Number of document chunks: {len(chunks)}")
print(f"Sample chunk: \n{chunks[3].page_content}\n")

Document chunk info:

Number of document chunks: 267
Sample chunk: 
A hemisphere is half of a sphere, but not exactly a half sphere. A sphere is a three-dimensional shape that is perfectly round and symmetrical all the way around. If you cut a sphere in half through its center, you would get a hemisphere.

Now, a half sphere is half of a sphere, but it's more like a soft drink cup or a bowl. It's not symmetrical all the way around. If you were to cut a half sphere in half again, you wouldn't get two identical hemispheres, because the original half sphere wasn't symmetrical to begin with.

In other words, a hemisphere is always exactly half of a sphere, but a half sphere isn't necessarily half of a sphere. Does that make sense?

Imagine a beach ball. If you cut it in half through its center, you'll get two hemispheres (they'll be identical). But if you cut it in half through the middle, but not through the center, you'll get two non-identical half spheres.



In [13]:
#create embeddings using openAI embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

In [14]:
#store the embeddings and chunks into Chroma DB
Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_dir)

Retrieve and generate

In [15]:
#setting up the DB for retrieval
embeddings_used = OpenAIEmbeddings(model="text-embedding-3-small")
vectorDB = Chroma(persist_directory=db_dir,embedding_function=embeddings_used)

In [16]:
#setting up Retriver
retriever = vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [17]:
def getRetriever(dir):
    """
    dir is the directory of the vector DB
    """
    embeddings_used = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorDB = Chroma(persist_directory=dir,embedding_function=embeddings_used)
    retriever = vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    return retriever

In [65]:
def storyRephrase_langChain_RAG(msg,theme,retrieverDir):
    
    llm = ChatOpenAI(
            model="gpt-4o",
            temperature=0.2,
            max_tokens=200,
            timeout=None,
            max_retries=2
        )

    system_prompt = (
        "You are a high school reading tutor to let students undersand the story {scenario_lang} better based around the theme of {theme}" 
        "Use the following pieces of retrieved context to rephrase the story {scenario_lang} to words about {theme}. Use the word {theme}"
        "Use simple sentences and simple words for students to understand better"
        "\n\n"
        "{context}"
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{scenario_lang}"),
        ]
    )

    rag_chain = prompt | llm | StrOutputParser()

    retriever = getRetriever(retrieverDir)

    out_message = rag_chain.invoke({
            "theme" : theme,
            "context":retriever,
            "scenario_lang" : msg,
        })
    
    return out_message

In [66]:
question = "I understand the view I suppose, but it seems to me that blatant discrimination on skin color is very rare, at least in my parts of the south. The greater stigmas are cultural and economic, and will be based on your own belief, class, and your childs name and friends. None of these will change because of the skin color of your partner." #example output from huggingface model
newQ = storyRephrase_langChain_RAG("basketball", question, db_dir)
print(newQ)

Basketball is a sport where two teams try to score points by throwing a ball into a hoop. It's a game that brings people together, regardless of their skin color. In some places, like the south, people believe that discrimination based on skin color is not common. Instead, people might face challenges because of their culture, how much money they have, or who they are friends with. These things are more important than the color of your skin. So, when you play basketball, remember that everyone is equal on the court, and what matters most is how you play and treat others.
